In [1]:
!pip3 install datasets transformers -q
!pip3 install wandb --upgrade -q

     |████████████████████████████████| 342 kB 9.9 MB/s 
     |████████████████████████████████| 4.2 MB 56.9 MB/s 
     |████████████████████████████████| 140 kB 53.0 MB/s 
     |████████████████████████████████| 84 kB 3.4 MB/s 
     |████████████████████████████████| 212 kB 51.7 MB/s 
     |████████████████████████████████| 1.1 MB 47.8 MB/s 
     |████████████████████████████████| 127 kB 52.5 MB/s 
     |████████████████████████████████| 596 kB 49.2 MB/s 
     |████████████████████████████████| 6.6 MB 35.5 MB/s 
     |████████████████████████████████| 94 kB 2.0 MB/s 
     |████████████████████████████████| 144 kB 53.7 MB/s 
     |████████████████████████████████| 271 kB 26.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 1.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
from sklearn.preprocessing import MultiLabelBinarizer

from datasets import load_dataset
import random
from sklearn import metrics, model_selection, preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup

In [3]:
torch.cuda.is_available()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

In [4]:
def seed_everything(seed=73):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # some cudnn methods can be random even after fixing the seed unless you tell it to be deterministic
    torch.backends.cudnn.deterministic = True

seed_everything(1234)

In [5]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
dataset = load_dataset('eurlex', 'eurlex57k')
data = dataset.data

Generating train split:   0%|          | 0/45000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6000 [00:00<?, ? examples/s]

Dataset eurlex downloaded and prepared to /root/.cache/huggingface/datasets/eurlex/eurlex57k/1.1.0/d2fdeaa4fcb5f41394d2ed0317c8541d7f9be85d2d601b9fa586c8b461bc3a34. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
train, val, test = data["train"].to_pandas(), data["validation"].to_pandas(), data["test"].to_pandas()
print(train.shape, val.shape, test.shape) 

(45000, 4) (6000, 4) (6000, 4)


In [8]:
train['text'] = train['title'] + " " + train['text']
val['text'] = val['title'] + " " + val['text']
test['text'] = test['title'] + " " + test['text']

In [9]:
train.head()

,celex_id,title,text,eurovoc_concepts
0,32014R0727,Commission Implementing Regulation (EU) No 727...,Commission Implementing Regulation (EU) No 727...,"[1402, 2771, 3191, 5055, 519, 5969, 5971]"
1,31975R2481,Regulation (EEC) No 2481/75 of the Council of ...,Regulation (EEC) No 2481/75 of the Council of ...,"[2319, 2713, 2938, 693]"
2,32010D0008,"2010/8/EU, Euratom: Commission Decision of 22 ...","2010/8/EU, Euratom: Commission Decision of 22 ...","[3560, 365, 4256, 4261, 4353, 4585]"
3,31982D0211,82/211/EEC: Commission Decision of 17 March 19...,82/211/EEC: Commission Decision of 17 March 19...,"[1091, 3842, 3874, 4110, 4381, 5287]"
4,31996D0084,"96/84/Euratom, ECSC, EC: Commission Decision o...","96/84/Euratom, ECSC, EC: Commission Decision o...","[1026, 1048, 2300, 3653, 4271, 4390]"


In [10]:
concepts = pd.read_json('dictionary.json').T['label']
mapping = concepts.to_dict()

In [11]:
import re
def ohe(dataset):
    one_hot = MultiLabelBinarizer()
    labels = dataset['eurovoc_concepts']
    ohe = pd.DataFrame(one_hot.fit_transform(labels).T, index=one_hot.classes_)
    ohe = pd.merge(concepts, ohe, left_index=True, right_index=True)
    ohe.index = ohe['label']
    ohe = ohe.drop('label', axis=1)
    ohe = ohe.T
#     ohe.columns = list(map(lambda x: indexing[x], ohe.columns.values))
    return ohe

def num_digits(s):
    return sum(c.isdigit() for c in s)

def clean_text(text):
    text = text.replace("\\", " ")
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', ' ', text)
    text = re.sub(
        '[.,:;_%©?*,!@#$%^&(){{}}]|[+=]|[«»]|[<>]|[\']|[[]|[]]|[/]|"|\s{2,}|-', ' ', text)
    text = ' '.join(word for word in text.split() if len(word) > 2)
    text = ' '.join(word for word in text.split() if not word.isnumeric())
    text = ' '.join(word for word in text.split() if num_digits(word)<=2)
    # text = " ".join(word for word in text.split() if word not in stop_words)
    text = ' '.join(text.split())
    return text

In [12]:
ohe_train = ohe(train)
ohe_val = ohe(val)
ohe_test = ohe(test)
all_documents = pd.concat([ohe_train, ohe_val, ohe_test]).fillna(0)
all_documents.columns = range(len(all_documents.columns))
ohe_train = all_documents.iloc[:45000, (all_documents.sum() > 500).values]
ohe_val = all_documents.iloc[45000:51000, (all_documents.sum() > 500).values]
ohe_test = all_documents.iloc[51000:, (all_documents.sum() > 500).values]

In [13]:
train = pd.concat([train.iloc[:, :], ohe_train], axis=1)
val = pd.concat([val, ohe_val], axis=1)
test = pd.concat([test, ohe_test], axis=1)
train['text'] = train['text'].apply(clean_text)
val['text'] = val['text'].apply(clean_text)
test['text'] = test['text'].apply(clean_text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: Possible nested set at position 46


In [14]:
del(ohe_train)
del(ohe_test)
del(ohe_val)
del(all_documents)

In [15]:
n_labels = len(train.columns) - 4
n_labels

121

In [16]:
class Dataset:
    def __init__(self, texts, labels, tokenizer, max_len=100):
        self.texts = texts
        self.labels = labels

        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        inputs = self.tokenizer.__call__(text,
                                        None,
                                        add_special_tokens=True,
                                        max_length=self.max_len,
                                        padding="max_length",
                                        truncation=True,
                                        )
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "labels": torch.tensor(label, dtype=torch.long)
        }

# def get_classes(data, min_class_ex=0, max_class_ex=50000):
#     ohe_data = one_hot_encoder(data)
#     indices = list(ohe_data.sum() > min_class_ex) * (ohe_data.sum() < max_class_ex)
#     return list(ohe_data.columns[indices])

# print(get_classes(train, 50))

In [17]:
# def get_data(train, val, test, min_class_ex=0,  max_class_ex=58000, classes=None):
#     """
#     if classes == None 
#         gets ohe_data for classes with <=max_class_ex, >=min_class_ex example per class (in train)
#     else
#         gets ohe_data for given classes
#     """
#     data = pd.concat([train, val, test], ignore_index=True).fillna(0)

#     if classes is None:
#         classes = get_classes(train, min_class_ex, max_class_ex)

#     n_classes = len(classes)

#     ohe_data = one_hot_encoder(data, classes)

#     # delete zero-label examples
#     # 
#     # zero_labels = np.array(ohe_data.sum(axis=1) < 1)
#     # zero_labels_idx = np.array(range(len(zero_labels)))[zero_labels]
#     # ohe_data = ohe_data.drop(zero_labels_idx).reset_index(drop=True)

#     # add new label = zero_lables
#     # 
#     ohe_data["zero labels"] = [0] * len(ohe_data)
#     zero_labels = np.array(ohe_data.sum(axis=1) < 1)
#     ohe_data["zero labels"][zero_labels] = 1

#     ohe_data.columns = list(range(n_classes + 1))
#     ohe_data.insert(0, "text", data["text"])

#     ohe_train = ohe_data.iloc[:45000]
#     ohe_val = ohe_data.iloc[45000:51000]
#     ohe_test = ohe_data.iloc[51000:]

#     return ohe_train, ohe_val, ohe_test, n_classes



# ex_train, _, _ , _ = get_data(train[:10], val[:10], test[:10], classes=['1026', '1048'])
# print(ex_train)



In [18]:
def rp_k_metric(preds, labels, k=5):
    idx = np.argpartition(preds, kth=-k, axis=-1)[:,-k:]
    div_k = np.clip(np.sum(labels, axis=1), 0, k)
    N = labels.shape[0]

    return np.sum(np.sum(np.take_along_axis(labels, idx, axis=-1), axis=-1) / div_k) / N

def ranking_rprecision_score(y_true, y_score, k=5):
    """Precision at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array-like, shape = [n_samples]
        Predicted scores.
    k : int
        Rank.
    Returns
    -------
    precision @k : float
    """
    unique_y = np.unique(y_true)

    if len(unique_y) == 1:
        return ValueError("The score cannot be approximated.")
    elif len(unique_y) > 2:
        raise ValueError("Only supported for two relevance levels.")

    pos_label = unique_y[1]
    n_pos = np.sum(y_true == pos_label)

    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    n_relevant = np.sum(y_true == pos_label)

    # Divide by min(n_pos, k) such that the best achievable score is always 1.0.
    return float(n_relevant) / min(k, n_pos)

def mean_rprecision_k(y_true, y_score, k=5):
    """Mean precision at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array-like, shape = [n_samples]
        Predicted scores.
    k : int
        Rank.
    Returns
    -------
    mean precision @k : float
    """

    p_ks = []
    for y_t, y_s in zip(y_true, y_score):
        if np.sum(y_t == 1):
            p_ks.append(ranking_rprecision_score(y_t, y_s, k=k))

    return np.mean(p_ks)


def log_metrics(preds, labels):
    preds = torch.stack(preds)
    preds = preds.cpu().detach().numpy()
    labels = torch.stack(labels)
    labels = labels.cpu().detach().numpy()
    
    precision_micro = metrics.average_precision_score(labels, preds, average="micro")
    rp_k = mean_rprecision_k(labels, preds, 5)

    fpr_micro, tpr_micro, _ = metrics.roc_curve(labels.ravel(), preds.ravel())
    auc_micro = metrics.auc(fpr_micro, tpr_micro)
    
    f1_score_micro = metrics.f1_score(labels, np.array(preds) > 0.2, average='micro')
    f1_score_macro = metrics.f1_score(labels, np.array(preds) > 0.2, average='macro')
    
    return {
        "precision_micro": precision_micro,
        "RP@K": rp_k,
        "auc": auc_micro,
        "f1_score_micro": f1_score_micro,
        "f1_score_macro": f1_score_macro
    }

In [25]:


sweep_config = {
    'method': 'random', #grid, random, bayesian
    'metric': {
      'name': 'RP@K',
      'goal': 'maximize'   
    },
    'parameters': {

        'learning_rate': {
            'values': [1e-4, 2e-4, 5e-5]
        },
        'batch_size': {
            'values': [32]
        },
        'epochs':{'value': 5},
        'dropout':{
            'values': [0.4]
        },
        'tokenizer_max_len': {'value': 40},
    }
}

sweep_defaults = {
    'learning_rate': 1e-4,
    'batch_size': 32,
    'epochs': 5,
    'dropout': 0.3,
    'tokenizer_max_len': 40
}

sweep_id = wandb.sweep(sweep_config, project='bhaavnaye')
# class Config():
#   def __init__(self):
#     self.batch_size = 32
#     self.epochs = 5
#     self.dropout = 0.4
#     self.tokenizer_max_len = 100

# sweep_defaults = Config()



Create sweep with ID: t7ic1iiu
Sweep URL: https://wandb.ai/troechka/bhaavnaye/sweeps/t7ic1iiu


In [20]:
def ret_optimizer(model):
    '''
    Taken from Abhishek Thakur's Tez library example: 
    https://github.com/abhishekkrthakur/tez/blob/main/examples/text_classification/binary.py
    '''
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]
    opt = AdamW(optimizer_parameters, lr=wandb.config.learning_rate) #wandb.config.learning_rate)
    return opt

def ret_scheduler(optimizer, num_train_steps):
    sch = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
    return sch

def loss_fn(outputs, labels):
    if labels is None:
        return None
    return nn.BCEWithLogitsLoss()(outputs, labels.float())



In [21]:


class Classifier(nn.Module):
    def __init__(self, n_train_steps, n_classes, do_prob, bert_model):
        super(Classifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(do_prob)
        self.out = nn.Linear(768, n_classes)
        self.n_train_steps = n_train_steps
        self.step_scheduler_after = "batch"

    def forward(self, ids, mask):
        output_1 = self.bert(ids, attention_mask=mask)["pooler_output"]
        output_2 = self.dropout(output_1)
        output = self.out(output_2)
        return output



tokenizer = transformers.SqueezeBertTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased", do_lower_case=True)

def build_dataset(tokenizer_max_len, train, val, n_labels):
    train_dataset = Dataset(train.text.tolist(), train.iloc[:, 4:].values.tolist(), tokenizer, 100)
    val_dataset = Dataset(val.text.tolist(), val.iloc[:, 4:].values.tolist(), tokenizer, 100)
    
    return train_dataset, val_dataset 

def build_dataloader(train_dataset, val_dataset, batch_size):
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

    return train_data_loader, val_data_loader

def ret_model(n_train_steps, do_prob, n_labels):
  model = Classifier(n_train_steps, n_labels, do_prob, bert_model=bert_model)
  return model

bert_model = transformers.SqueezeBertModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

Downloading:   0%|          | 0.00/217k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'SqueezeBertTokenizer'.
You are using a model of type bert to instantiate a model of type squeezebert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing SqueezeBertModel: ['bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.6.output.dense.bias', 'bert.encoder.layer.7.intermediate.dense.weight', 'bert.encoder.layer.2.output.dense.weight', 'bert.encoder.layer.4.output.LayerNorm.bias', 'bert.encoder.layer.10.attention.self.key.bias', 'bert.encoder.layer.7.attention.self.query.weight', 'bert.encoder.layer.10.attention.output.LayerNorm.bias', 'bert.encoder.layer.9.attention.self.key.bias', 'bert.pooler.dense.weight', 'bert.encoder.layer.1.attention.self.query.weight', 'bert.encoder.layer.3.attention.self.value.weight', 'bert.encoder.layer.11.intermediate.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.encoder.layer.3.attention.output.dense.bias', 'bert.encoder.layer.7.output.dense.bias', 'bert.encoder.layer.4.intermediate.dense.weight', 'bert.encoder.layer.4.attention.output.dense.weight', 'bert.enco

In [22]:
test_dataset = Dataset(test.text.tolist(), test.iloc[:, 4:].values.tolist(), tokenizer, 100)
test_data_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=1)


In [23]:
def train_fn(data_loader, model, optimizer, device, scheduler):
    '''
        Modified from Abhishek Thakur's BERT example: 
        https://github.com/abhishekkrthakur/bert-sentiment/blob/master/src/engine.py
    '''

    train_loss = 0.0
    model.train()
    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d["ids"]
        mask = d["mask"]
        targets = d["labels"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask)

        loss = loss_fn(outputs, targets)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        scheduler.step()
    return train_loss
    

def eval_fn(data_loader, model, device):
    '''
        Modified from Abhishek Thakur's BERT example: 
        https://github.com/abhishekkrthakur/bert-sentiment/blob/master/src/engine.py
    '''
    eval_loss = 0.0
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            mask = d["mask"]
            targets = d["labels"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(ids=ids, mask=mask)
            loss = loss_fn(outputs, targets)
            eval_loss += loss.item()
            wandb.log({
                "train_loss": loss.item()
            })
            fin_targets.extend(targets)
            fin_outputs.extend(torch.sigmoid(outputs))
    return eval_loss, fin_outputs, fin_targets


def trainer(config=None):
    with wandb.init(config=config):
        config = wandb.config #sweep_defaults

        train_dataset, val_dataset = build_dataset(100, train, val, n_labels)
        train_data_loader, val_data_loader = build_dataloader(train_dataset, val_dataset, config.batch_size)
        print("Length of Train Dataloader: ", len(train_data_loader))
        print("Length of Valid Dataloader: ", len(val_data_loader))

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        n_train_steps = int(len(train_dataset) / config.batch_size * 10)

        model = ret_model(n_train_steps, config.dropout, n_labels)
        optimizer = ret_optimizer(model)
        scheduler = ret_scheduler(optimizer, n_train_steps)
        model.to(device)
        model = nn.DataParallel(model)
        wandb.watch(model)
        
        n_epochs = config.epochs

        best_val_loss = 100
        for epoch in tqdm(range(n_epochs)):
            train_loss = train_fn(train_data_loader, model, optimizer, device, scheduler)
            eval_loss, preds, labels = eval_fn(val_data_loader, model, device)
          
            print(log_metrics(preds, labels))

            # # check first 5 labels:
            # print(np.argpartition(torch.stack(preds).cpu().detach().numpy(), kth=-5, axis=-1)[:20,-5:])

            avg_train_loss, avg_val_loss = train_loss / len(train_data_loader), eval_loss / len(val_data_loader)
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": avg_train_loss,
                "val_loss": avg_val_loss
            })
            print("Average Train loss: ", avg_train_loss)
            print("Average Valid loss: ", avg_val_loss)

            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                torch.save(model.state_dict(), "./best_model.pt")  
                print("Model saved as current val_loss is: ", best_val_loss)

In [ ]:
wandb.agent(sweep_id, function=trainer, count=6)

wandb: Agent Starting Run: 4xlafuii with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.4
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0002
wandb: 	tokenizer_max_len: 40
wandb: Currently logged in as: troechka. Use `wandb login --relogin` to force relogin


Length of Train Dataloader:  1407
Length of Valid Dataloader:  188


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1407 [00:00<?, ?it/s]

...

wandb: Agent Starting Run: 0m3l32tl with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.4
wandb: 	epochs: 5
wandb: 	learning_rate: 5e-05
wandb: 	tokenizer_max_len: 40


Length of Train Dataloader:  1407
Length of Valid Dataloader:  188


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1407 [00:00<?, ?it/s]

# New Section